In [3]:
import requests
import json
import psycopg2

# API details
url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json"
headers = {
    "accept": "application/json",
    "x-api-key": "lCd20Zc6VmbsEyIiMlPso3VOaMzgFxB3wC8k8s3Q"
}

# Request and parse JSON
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raise exception if status is not 200
data = response.json()

# Prepare data
complex_list = []
venue_list = []

for com in data.get('complexes', []):
    complex_list.append({
        'complex_id': com['id'],
        'complex_name': com['name'],
    })

    for venue in com.get('venues', []):
        venue_list.append({
            'venue_id': venue.get('id'),
            'venue_name': venue.get('name'),
            'city_name': venue.get('city_name'),
            'country_name': venue.get('country_name'),
            'country_code': venue.get('country_code'),
            'timezone': venue.get('timezone'),
            'complex_id': com['id']  # Fixed from com['complex']['id']
        })

# DB connection and insert
try:
    conn = psycopg2.connect(
        host="localhost",
        database="sports_radar",
        user="postgres",
        password="ASkl1289@"
    )
    cur = conn.cursor()

    # Insert complexes
    for comp in complex_list:
        cur.execute(
            """
            INSERT INTO sports_radar_schema.complexes (complex_id, complex_name)
            VALUES (%s, %s)
            ON CONFLICT (complex_id) DO NOTHING
            """,
            (comp['complex_id'], comp['complex_name'])
        )

    # Insert venues
    for venue in venue_list:
        cur.execute(
            """
            INSERT INTO sports_radar_schema.venues
            (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (venue_id) DO NOTHING
            """,
            (
                venue['venue_id'],
                venue['venue_name'],
                venue['city_name'],
                venue['country_name'],
                venue['country_code'],
                venue['timezone'],
                venue['complex_id']
            )
        )

    conn.commit()
    print("Data inserted successfully.")

except Exception as e:
    print("Database operation failed:", e)

finally:
    if conn:
        cur.close()
        conn.close()


Data inserted successfully.
